# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # Split text into sentences
    source_text_lst = source_text.split('\n')
    target_text_lst = target_text.split('\n')
    # Append <EOS> at the end of each sententence
    target_text_lst = [sentence + ' <EOS>' for sentence in target_text_lst]
    # Make lists using vocab to int mapping
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text_lst]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_text_lst]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


C:\Users\georg\Anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    inputs = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None))
    learn_rate = tf.placeholder(tf.float32, shape=None)
    keep_prob = tf.placeholder(tf.float32, shape=None, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return inputs, targets, learn_rate, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    drop = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
    
    enc_output, enc_state = tf.nn.dynamic_rnn(drop, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    
    # Helper for the training process. Used by BasicDecoder to read inputs.
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)
        
        
    # Basic decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 
        
    # Perform dynamic decoding using the decoder
    training_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                   impute_finished=True,
                                                                   maximum_iterations=max_summary_length)    
    
    return training_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # Reuses the same parameters trained by the training process
    
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')

    # Helper for the inference process.
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)

    # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
    # Perform dynamic decoding using the decoder
    inference_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                    impute_finished=True,
                                                                    maximum_iterations=max_target_sequence_length)
         
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [14]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Embed the target sequences
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_inputs = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # Construct the decoder LSTM cell (just like you constructed the encoder cell above)
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])

    # Create an output layer to map the outputs of the decoder to the elements of our vocabulary
    output_layer = Dense(target_vocab_size, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))



    with tf.variable_scope("decode"):

        # Helper for the training process. Used by BasicDecoder to read inputs.
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_inputs,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        
        
        # Basic decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 
        
        # Perform dynamic decoding using the decoder
        training_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_sequence_length)
    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=True):
        start_tokens = tf.tile(tf.constant([target_vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')

        # Helper for the inference process.
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                target_vocab_to_int['<EOS>'])

        # Basic decoder
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
        # Perform dynamic decoding using the decoder
        inference_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)
         

    
    return training_decoder_output, inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [18]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers,
                                  keep_prob,
                                  source_sequence_length,
                                  source_vocab_size, 
                                  enc_embedding_size)
    
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input,
                                                                       enc_state,
                                                                       target_sequence_length,
                                                                       max_target_sentence_length,
                                                                       rnn_size,
                                                                       num_layers,
                                                                       target_vocab_to_int, 
                                                                       target_vocab_size,
                                                                       batch_size,
                                                                       keep_prob,
                                                                       dec_embedding_size) 
    
    return training_decoder_output, inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [19]:
# Number of Epochs
epochs = 30
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 13
decoding_embedding_size = 13
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.8
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/1077 - Train Accuracy: 0.1982, Validation Accuracy: 0.3050, Loss: 5.6915
Epoch   0 Batch   20/1077 - Train Accuracy: 0.2316, Validation Accuracy: 0.3050, Loss: 4.8823
Epoch   0 Batch   30/1077 - Train Accuracy: 0.2277, Validation Accuracy: 0.3050, Loss: 4.1802
Epoch   0 Batch   40/1077 - Train Accuracy: 0.2434, Validation Accuracy: 0.3185, Loss: 3.8007
Epoch   0 Batch   50/1077 - Train Accuracy: 0.2562, Validation Accuracy: 0.3349, Loss: 3.6376
Epoch   0 Batch   60/1077 - Train Accuracy: 0.2958, Validation Accuracy: 0.3356, Loss: 3.3752
Epoch   0 Batch   70/1077 - Train Accuracy: 0.2294, Validation Accuracy: 0.3363, Loss: 3.5404
Epoch   0 Batch   80/1077 - Train Accuracy: 0.2711, Validation Accuracy: 0.3363, Loss: 3.3012
Epoch   0 Batch   90/1077 - Train Accuracy: 0.2668, Validation Accuracy: 0.3363, Loss: 3.3037
Epoch   0 Batch  100/1077 - Train Accuracy: 0.2617, Validation Accuracy: 0.3363, Loss: 3.2706
Epoch   0 Batch  110/1077 - Train Accuracy: 0.2836, Validati

Epoch   0 Batch  890/1077 - Train Accuracy: 0.5134, Validation Accuracy: 0.4876, Loss: 1.4721
Epoch   0 Batch  900/1077 - Train Accuracy: 0.4547, Validation Accuracy: 0.4833, Loss: 1.6623
Epoch   0 Batch  910/1077 - Train Accuracy: 0.4654, Validation Accuracy: 0.4936, Loss: 1.5440
Epoch   0 Batch  920/1077 - Train Accuracy: 0.4500, Validation Accuracy: 0.4830, Loss: 1.6009
Epoch   0 Batch  930/1077 - Train Accuracy: 0.4594, Validation Accuracy: 0.4826, Loss: 1.5530
Epoch   0 Batch  940/1077 - Train Accuracy: 0.4418, Validation Accuracy: 0.4844, Loss: 1.6069
Epoch   0 Batch  950/1077 - Train Accuracy: 0.4490, Validation Accuracy: 0.4766, Loss: 1.5465
Epoch   0 Batch  960/1077 - Train Accuracy: 0.4940, Validation Accuracy: 0.4993, Loss: 1.4574
Epoch   0 Batch  970/1077 - Train Accuracy: 0.4641, Validation Accuracy: 0.4936, Loss: 1.5560
Epoch   0 Batch  980/1077 - Train Accuracy: 0.4816, Validation Accuracy: 0.4975, Loss: 1.4804
Epoch   0 Batch  990/1077 - Train Accuracy: 0.4453, Validati

Epoch   1 Batch  700/1077 - Train Accuracy: 0.4426, Validation Accuracy: 0.5043, Loss: 0.8986
Epoch   1 Batch  710/1077 - Train Accuracy: 0.4641, Validation Accuracy: 0.5064, Loss: 0.9194
Epoch   1 Batch  720/1077 - Train Accuracy: 0.4688, Validation Accuracy: 0.5114, Loss: 1.0010
Epoch   1 Batch  730/1077 - Train Accuracy: 0.4965, Validation Accuracy: 0.5188, Loss: 0.9248
Epoch   1 Batch  740/1077 - Train Accuracy: 0.5094, Validation Accuracy: 0.5433, Loss: 0.8966
Epoch   1 Batch  750/1077 - Train Accuracy: 0.5012, Validation Accuracy: 0.5387, Loss: 0.9004
Epoch   1 Batch  760/1077 - Train Accuracy: 0.4797, Validation Accuracy: 0.5373, Loss: 0.9233
Epoch   1 Batch  770/1077 - Train Accuracy: 0.5126, Validation Accuracy: 0.5401, Loss: 0.8659
Epoch   1 Batch  780/1077 - Train Accuracy: 0.4832, Validation Accuracy: 0.5380, Loss: 0.9320
Epoch   1 Batch  790/1077 - Train Accuracy: 0.4406, Validation Accuracy: 0.5281, Loss: 0.9526
Epoch   1 Batch  800/1077 - Train Accuracy: 0.4566, Validati

Epoch   2 Batch  510/1077 - Train Accuracy: 0.5957, Validation Accuracy: 0.6030, Loss: 0.6966
Epoch   2 Batch  520/1077 - Train Accuracy: 0.6198, Validation Accuracy: 0.6104, Loss: 0.6737
Epoch   2 Batch  530/1077 - Train Accuracy: 0.5707, Validation Accuracy: 0.6065, Loss: 0.7319
Epoch   2 Batch  540/1077 - Train Accuracy: 0.5762, Validation Accuracy: 0.6112, Loss: 0.6788
Epoch   2 Batch  550/1077 - Train Accuracy: 0.5414, Validation Accuracy: 0.6069, Loss: 0.7500
Epoch   2 Batch  560/1077 - Train Accuracy: 0.5734, Validation Accuracy: 0.5952, Loss: 0.6986
Epoch   2 Batch  570/1077 - Train Accuracy: 0.5950, Validation Accuracy: 0.6062, Loss: 0.7367
Epoch   2 Batch  580/1077 - Train Accuracy: 0.6168, Validation Accuracy: 0.6094, Loss: 0.6508
Epoch   2 Batch  590/1077 - Train Accuracy: 0.5814, Validation Accuracy: 0.6058, Loss: 0.7323
Epoch   2 Batch  600/1077 - Train Accuracy: 0.5893, Validation Accuracy: 0.6104, Loss: 0.6671
Epoch   2 Batch  610/1077 - Train Accuracy: 0.5757, Validati

Epoch   3 Batch  320/1077 - Train Accuracy: 0.6438, Validation Accuracy: 0.6381, Loss: 0.5973
Epoch   3 Batch  330/1077 - Train Accuracy: 0.6594, Validation Accuracy: 0.6346, Loss: 0.5877
Epoch   3 Batch  340/1077 - Train Accuracy: 0.5933, Validation Accuracy: 0.6360, Loss: 0.5904
Epoch   3 Batch  350/1077 - Train Accuracy: 0.5922, Validation Accuracy: 0.6296, Loss: 0.6133
Epoch   3 Batch  360/1077 - Train Accuracy: 0.5930, Validation Accuracy: 0.6431, Loss: 0.5847
Epoch   3 Batch  370/1077 - Train Accuracy: 0.6235, Validation Accuracy: 0.6282, Loss: 0.5544
Epoch   3 Batch  380/1077 - Train Accuracy: 0.6051, Validation Accuracy: 0.6417, Loss: 0.5742
Epoch   3 Batch  390/1077 - Train Accuracy: 0.5914, Validation Accuracy: 0.6392, Loss: 0.6077
Epoch   3 Batch  400/1077 - Train Accuracy: 0.6270, Validation Accuracy: 0.6420, Loss: 0.5935
Epoch   3 Batch  410/1077 - Train Accuracy: 0.5843, Validation Accuracy: 0.6445, Loss: 0.5977
Epoch   3 Batch  420/1077 - Train Accuracy: 0.6156, Validati

...

Epoch  13 Batch  180/1077 - Train Accuracy: 0.8793, Validation Accuracy: 0.8533, Loss: 0.0901
Epoch  13 Batch  190/1077 - Train Accuracy: 0.9062, Validation Accuracy: 0.8374, Loss: 0.0852
Epoch  13 Batch  200/1077 - Train Accuracy: 0.8914, Validation Accuracy: 0.8501, Loss: 0.0977
Epoch  13 Batch  210/1077 - Train Accuracy: 0.8754, Validation Accuracy: 0.8391, Loss: 0.0955
Epoch  13 Batch  220/1077 - Train Accuracy: 0.8976, Validation Accuracy: 0.8594, Loss: 0.0865
Epoch  13 Batch  230/1077 - Train Accuracy: 0.8802, Validation Accuracy: 0.8526, Loss: 0.0885
Epoch  13 Batch  240/1077 - Train Accuracy: 0.9070, Validation Accuracy: 0.8530, Loss: 0.0866
Epoch  13 Batch  250/1077 - Train Accuracy: 0.8786, Validation Accuracy: 0.8679, Loss: 0.0768
Epoch  13 Batch  260/1077 - Train Accuracy: 0.9051, Validation Accuracy: 0.8487, Loss: 0.0780
Epoch  13 Batch  270/1077 - Train Accuracy: 0.8426, Validation Accuracy: 0.8775, Loss: 0.1107
Epoch  13 Batch  280/1077 - Train Accuracy: 0.8598, Validati

Epoch  13 Batch 1060/1077 - Train Accuracy: 0.8820, Validation Accuracy: 0.8764, Loss: 0.0667
Epoch  13 Batch 1070/1077 - Train Accuracy: 0.8559, Validation Accuracy: 0.8604, Loss: 0.0795
Epoch  14 Batch   10/1077 - Train Accuracy: 0.9017, Validation Accuracy: 0.8686, Loss: 0.0841
Epoch  14 Batch   20/1077 - Train Accuracy: 0.8855, Validation Accuracy: 0.8793, Loss: 0.0664
Epoch  14 Batch   30/1077 - Train Accuracy: 0.9184, Validation Accuracy: 0.8604, Loss: 0.0747
Epoch  14 Batch   40/1077 - Train Accuracy: 0.8988, Validation Accuracy: 0.9006, Loss: 0.0701
Epoch  14 Batch   50/1077 - Train Accuracy: 0.9227, Validation Accuracy: 0.8615, Loss: 0.0728
Epoch  14 Batch   60/1077 - Train Accuracy: 0.8787, Validation Accuracy: 0.8736, Loss: 0.0755
Epoch  14 Batch   70/1077 - Train Accuracy: 0.8812, Validation Accuracy: 0.8775, Loss: 0.0925
Epoch  14 Batch   80/1077 - Train Accuracy: 0.8945, Validation Accuracy: 0.8608, Loss: 0.0769
Epoch  14 Batch   90/1077 - Train Accuracy: 0.8742, Validati

Epoch  14 Batch  870/1077 - Train Accuracy: 0.8388, Validation Accuracy: 0.8878, Loss: 0.0842
Epoch  14 Batch  880/1077 - Train Accuracy: 0.8902, Validation Accuracy: 0.8832, Loss: 0.0756
Epoch  14 Batch  890/1077 - Train Accuracy: 0.8940, Validation Accuracy: 0.8860, Loss: 0.0751
Epoch  14 Batch  900/1077 - Train Accuracy: 0.9223, Validation Accuracy: 0.8796, Loss: 0.0726
Epoch  14 Batch  910/1077 - Train Accuracy: 0.9062, Validation Accuracy: 0.8928, Loss: 0.0727
Epoch  14 Batch  920/1077 - Train Accuracy: 0.8953, Validation Accuracy: 0.8778, Loss: 0.0663
Epoch  14 Batch  930/1077 - Train Accuracy: 0.9004, Validation Accuracy: 0.8931, Loss: 0.0633
Epoch  14 Batch  940/1077 - Train Accuracy: 0.8930, Validation Accuracy: 0.8722, Loss: 0.0622
Epoch  14 Batch  950/1077 - Train Accuracy: 0.8769, Validation Accuracy: 0.8928, Loss: 0.0757
Epoch  14 Batch  960/1077 - Train Accuracy: 0.9010, Validation Accuracy: 0.8778, Loss: 0.0645
Epoch  14 Batch  970/1077 - Train Accuracy: 0.9273, Validati

Epoch  15 Batch  680/1077 - Train Accuracy: 0.8895, Validation Accuracy: 0.8995, Loss: 0.0715
Epoch  15 Batch  690/1077 - Train Accuracy: 0.9012, Validation Accuracy: 0.8974, Loss: 0.0636
Epoch  15 Batch  700/1077 - Train Accuracy: 0.9258, Validation Accuracy: 0.9105, Loss: 0.0578
Epoch  15 Batch  710/1077 - Train Accuracy: 0.8992, Validation Accuracy: 0.8991, Loss: 0.0608
Epoch  15 Batch  720/1077 - Train Accuracy: 0.8890, Validation Accuracy: 0.9006, Loss: 0.0762
Epoch  15 Batch  730/1077 - Train Accuracy: 0.9098, Validation Accuracy: 0.8849, Loss: 0.0798
Epoch  15 Batch  740/1077 - Train Accuracy: 0.9117, Validation Accuracy: 0.8999, Loss: 0.0572
Epoch  15 Batch  750/1077 - Train Accuracy: 0.9289, Validation Accuracy: 0.8800, Loss: 0.0707
Epoch  15 Batch  760/1077 - Train Accuracy: 0.8918, Validation Accuracy: 0.8935, Loss: 0.0713
Epoch  15 Batch  770/1077 - Train Accuracy: 0.8850, Validation Accuracy: 0.9059, Loss: 0.0679
Epoch  15 Batch  780/1077 - Train Accuracy: 0.8863, Validati

KeyboardInterrupt: 

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [ ]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

## Translate
This will translate `translate_sentence` from English to French.

In [ ]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.